## Package ml-recommendation (org.apache.spark.ml.recommendation)

L'objectif est de faire une présentation de la librairie de recommendation de Spark ML (Scala).

**Note :** Je mets plutôt l'accent sur la démarche et non sur la recherche d'un meilleur modèle.

### 1. Factorisation  de matrice

La factorisation de matrice est l'une des techniques les plus utilisées pour construire un système de recommandation. Il s'agit de décomposer une matrice en un produit de matrices. En d'autres termes, il faut retrouver des matrices dont le produit est une approximation de la matrice à factoriser. Dans Spark ce problème de minimisation est résolu par l’algorithme itératif Alternating Least Squares (**ALS**).

## 2. Recommandation de livres

In [2]:
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS
import scala.collection.mutable.WrappedArray
import org.apache.spark.sql.Row

// Charger les données
val ratingsDF = spark.read.option("header",  true)
                   .option("inferSchema",  true)
                   .option("delimiter", ",").csv("../data/ratings.txt")

ratingsDF.show(3)

+-------+-------+------+
|user_id|book_id|rating|
+-------+-------+------+
|      1|    258|     5|
|      2|   4081|     4|
|      2|    260|     5|
+-------+-------+------+
only showing top 3 rows



import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.recommendation.ALS
import scala.collection.mutable.WrappedArray
import org.apache.spark.sql.Row
ratingsDF: org.apache.spark.sql.DataFrame = [user_id: int, book_id: int ... 1 more field]


In [3]:
ratingsDF.count

res1: Long = 5976479


In [4]:
// Séparer les données en train et test
val Array(training, test) = ratingsDF.randomSplit(Array(0.8, 0.2), seed = 34512)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: int, book_id: int ... 1 more field]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: int, book_id: int ... 1 more field]


In [5]:
// Algo ALS
val als = new ALS()
  .setUserCol("user_id")
  .setItemCol("book_id")
  .setRatingCol("rating")
  .setSeed(12345)
  .setNonnegative(true)  // Ajouter une contrainte pour avoir des matrices non négatives.
  .setImplicitPrefs(false) // Si les infos ont été recuillies de façon implite (achats, clics, etc..). 
  .setColdStartStrategy("drop") // l'erreur d'ajustement de la factorisation ne prend pas en compte les NA.

// Calcul du RMSE sur le test
val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setLabelCol("rating")
  .setPredictionCol("prediction")

// Grille de recherche pour trouver le meilleur modèle
val paramGrid = new ParamGridBuilder()
  .addGrid(als.rank, Array(5, 15, 25, 100)) // nombre de facteurs latents
  .addGrid(als.regParam,  Array(0.1, 0.3, 0.7)) // paramètre de régularisation pour la méthode ALS.
  .addGrid(als.maxIter, Array(5, 20, 40, 60)) // Nombre d'itérations
  .build()

als: org.apache.spark.ml.recommendation.ALS = als_613446f4fea0
evaluator: org.apache.spark.ml.evaluation.RegressionEvaluator = RegressionEvaluator: uid=regEval_592a755b91c1, metricName=rmse, throughOrigin=false
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	als_613446f4fea0-maxIter: 5,
	als_613446f4fea0-rank: 5,
	als_613446f4fea0-regParam: 0.1
}, {
	als_613446f4fea0-maxIter: 5,
	als_613446f4fea0-rank: 15,
	als_613446f4fea0-regParam: 0.1
}, {
	als_613446f4fea0-maxIter: 5,
	als_613446f4fea0-rank: 25,
	als_613446f4fea0-regParam: 0.1
}, {
	als_613446f4fea0-maxIter: 5,
	als_613446f4fea0-rank: 100,
	als_613446f4fea0-regParam: 0.1
}, {
	als_613446f4fea0-maxIter: 5,
	als_613446f4fea0-rank: 5,
	als_613446f4fea0-regParam: 0.3
}, {
	als_613446f4fea0-maxIter...


In [6]:
val trainValidationSplit = new TrainValidationSplit()
  .setEstimator(als)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(paramGrid)
  .setTrainRatio(0.8)
  .setParallelism(2)

// Fitting du modèle
val model = als.fit(training)

// Prediction en test
val predictions = model.transform(test)

predictions.select("rating", "prediction").show(5)

// Calcule du RMSE sur le test
val rmse = evaluator.evaluate(predictions)

print(s"RMSE = ${rmse}")

+------+----------+
|rating|prediction|
+------+----------+
|     3| 3.3020227|
|     4| 3.6534286|
|     4| 2.8139138|
|     5| 3.9554446|
|     2| 3.0176945|
+------+----------+
only showing top 5 rows

RMSE = 0.8332195327770032

trainValidationSplit: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_643b30fd3338
model: org.apache.spark.ml.recommendation.ALSModel = ALSModel: uid=als_613446f4fea0, rank=10
predictions: org.apache.spark.sql.DataFrame = [user_id: int, book_id: int ... 2 more fields]
rmse: Double = 0.8332195327770032


**Générer le top 3 des livres à recommender pour chaque utilisateur**

In [7]:
val userRecs = model.recommendForAllUsers(3)
userRecs.show(10)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|    148|[[3628, 4.344607]...|
|    463|[[8946, 4.1403933...|
|    471|[[8946, 4.3629804...|
|    496|[[6089, 5.020456]...|
|    833|[[8946, 4.76129],...|
|   1088|[[8946, 4.9013343...|
|   1238|[[1935, 4.62988],...|
|   1342|[[8946, 4.612919]...|
|   1580|[[3628, 4.6657906...|
|   1591|[[3628, 4.700842]...|
+-------+--------------------+
only showing top 10 rows



userRecs: org.apache.spark.sql.DataFrame = [user_id: int, recommendations: array<struct<book_id:int,rating:float>>]


**Générer pour chaque livre, le top 3 des utilisateurs peuvent être intéressés**

In [8]:
val booksRecs = model.recommendForAllItems(3)
booksRecs.show(10)

+-------+--------------------+
|book_id|     recommendations|
+-------+--------------------+
|   1580|[[41383, 4.762112...|
|   4900|[[51626, 4.818962...|
|   5300|[[52237, 4.730690...|
|   6620|[[3556, 5.032635]...|
|   7240|[[9514, 5.3250403...|
|   7340|[[32722, 4.886296...|
|   7880|[[38076, 4.88214]...|
|   9900|[[22895, 5.19854]...|
|    471|[[44751, 4.813693...|
|   1591|[[29106, 5.019598...|
+-------+--------------------+
only showing top 10 rows



booksRecs: org.apache.spark.sql.DataFrame = [book_id: int, recommendations: array<struct<user_id:int,rating:float>>]


**Générer le top 3 des livres à recommander pour une liste d'utilisateurs**

In [9]:
val users = ratingsDF.select(als.getUserCol).distinct().limit(3)
val userSubsetRecs = model.recommendForUserSubset(users, 3)
userSubsetRecs.collect().foreach {
      case Row(user_id: Int, recommendations : WrappedArray[_]) 
                => {
                    println(s"Recommandations pour user ID $user_id =>") 
                    recommendations.asInstanceOf[Seq[Array[_]]].map(
                       x => println(s"book_id, rating = ${x}"))
                }
}

Recommandations pour user ID 1580 =>
book_id, rating = [3628,4.6657906]
book_id, rating = [9578,4.65385]
book_id, rating = [8946,4.593843]
Recommandations pour user ID 463 =>
book_id, rating = [8946,4.1403933]
book_id, rating = [2082,4.03369]
book_id, rating = [4868,4.0255046]
Recommandations pour user ID 1238 =>
book_id, rating = [1935,4.62988]
book_id, rating = [9094,4.3564425]
book_id, rating = [8854,4.3408766]


users: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [user_id: int]
userSubsetRecs: org.apache.spark.sql.DataFrame = [user_id: int, recommendations: array<struct<book_id:int,rating:float>>]


**Générer le top 3 des utilisateurs qui peuvent être intéressés pour une liste de livre**

In [10]:
val books = ratingsDF.select(als.getItemCol).distinct().limit(3)
val booksSubSetRecs = model.recommendForItemSubset(books, 3)

booksSubSetRecs.collect().foreach {
      case Row(book_id: Int, recommendations : WrappedArray[_]) 
                => {
                    println(s"Livre $book_id:") 
                    println("Recommendations : =>")
                   recommendations.asInstanceOf[Seq[Array[_]]].map(
                       x => println(s"user_id, rating = ${x}"))
                }
}

Livre 471:
Recommendations : =>
user_id, rating = [44751,4.813693]
user_id, rating = [44678,4.7945585]
user_id, rating = [52237,4.7913785]
Livre 2142:
Recommendations : =>
user_id, rating = [32574,5.0247736]
user_id, rating = [27472,5.0161223]
user_id, rating = [46749,5.002241]
Livre 148:
Recommendations : =>
user_id, rating = [9514,4.829982]
user_id, rating = [8377,4.7716002]
user_id, rating = [29742,4.7560544]


books: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [book_id: int]
booksSubSetRecs: org.apache.spark.sql.DataFrame = [book_id: int, recommendations: array<struct<user_id:int,rating:float>>]


**Références :**   

[Documentation Spark](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)      
[WikiStat](https://github.com/wikistat/AI-Frameworks/blob/master/RecomendationSystem/pyspark.ipynb)   